In [2]:
import gensim

C:\Users\lappy\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:

word2vec = gensim.models.KeyedVectors.load_word2vec_format("C:/Users/lappy/Downloads/Qoura compettition/GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv("C:/Users/lappy/Downloads/Qoura compettition/train.csv")

In [ ]:
data.question_text.map(len).max()

In [ ]:
data.head()

In [ ]:
textdata=data['question_text'].str.lower()
labels=data['target']

In [ ]:
from keras.utils import to_categorical

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS=200000

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(textdata)
sequences = tokenizer.texts_to_sequences(textdata)

word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

x_traindata = pad_sequences(sequences, maxlen=200)

y_labels = to_categorical(np.asarray(labels))


In [ ]:
model=word2vec
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = model.wv[model.wv.index2word[i]]
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding


from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=200, trainable=False)
model.add(e)
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(300))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_traindata, labels, epochs=1, verbose=2)
# evaluate the model
df=pd.read_csv("../input/test.csv")
testd=df['question_text'].str.lower()
MAX_NB_WORDS=200000

tokenizer2 = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer2.fit_on_texts(textdata)
sequences = tokenizer2.texts_to_sequences(testd)
x_testdata = pad_sequences(sequences, maxlen=200)
prediction = model.predict_classes(x_testdata)
qid=df['qid']
prediction=np.reshape(prediction,(prediction.size))
df = pd.DataFrame({"qid" : qid, "prediction" : prediction})
df.to_csv("submission.csv", index=False)


embedding_matrix = np.zeros((len(model.wv.vocab) + 1, vector_dim=300))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector